### House Hacking Calculator

**Use this notebook to identify potential deals for house hacks**


In [1]:
### IMPORTS ###

from pywebio.input import input, input_group, select, NUMBER, TEXT
import pandas as pd

print('DONE')


DONE


In [2]:
### Establish Constants ###

# MAX_MONTHLY_PAYMENT = 3000
MAX_UPFRONT_INVENTMENT = 30500
CURRENT_MONTHLY_DEBT = 312
CURRENT_MONHTLY_NET_INCOME = 5000
CURRENT_MONHTLY_GROSS_INCOME = 6000
CURRENT_MONTHLY_RENT = 2500

print('DONE')

DONE


In [3]:
### ASSIGN VARIABLES ###

info = input_group('House Information', [
    input('House Price: ', name='price', required=True, type=NUMBER),
    input('House Age (yr): ', name='age', required=True, type=NUMBER),
    input('Number of rooms: ', name='rooms', required=True, type=NUMBER),
    input('HOA Fees: ', name='hoa', type=NUMBER),
    input('Estimated Principal and Interest Monthly: ', name='p_and_i', required=True, type=NUMBER),
    input('Estimated Up-front Renovation Cost: ', name='upfront_renovation_costs', type=NUMBER),
    select('Property Type: ', name='type', options=['Duplex', 'Triplex', 'Fourplex', 'Single-Family'], required=True, type=TEXT),
    select('Location Grade: ', name='location', options=['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D'], required=True, type=TEXT)
])

info['rent_by_room'] = input_group("Rent by Room: ", [input(f'Room {i+1}: ', name=f'room{i+1}', required=True, type=NUMBER) for i in range(info['rooms'])])

print('DONE')


DONE


In [ ]:
info

In [4]:
### FINANCING OPTIONS ###

# Initialize DataFrame
df = pd.DataFrame()

# Determine Eligablility
financing_options = {
    'fha': 0.035,
    '203k': 0.035,
    # 203K should do something dif with renovation upfron costs...
    'conventional_min': 0.05,
    'conventional_min_assuming_boarder_income': 0.05,
    'conventional_max': ((MAX_UPFRONT_INVENTMENT - info['upfront_renovation_costs']) / info['price']),
    'conventional_max_assuming_boarder_income': ((MAX_UPFRONT_INVENTMENT - info['upfront_renovation_costs']) / info['price'])
}

# Determine monthly reserved amount
letter_grades = ['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D']
for i in range(len(letter_grades)):
    if info['location'] == letter_grades[i]:
        location_aspect = i * 25
        if info['age'] <= 20:
            age_aspect = 0
        else:
            age_aspect = (info['age'] - 20)
        reserves = location_aspect + age_aspect + 200

index = 0
for option, percent in financing_options.items():
    df.at[index, 'financing_option'] = option
    df.at[index, 'down_payment_percentage'] = percent
    down_payment = info['price'] * percent
    df.at[index, 'down_payment'] = down_payment
    if info['price'] <= 625000 and percent >= 0.05:
        PMI = 0.008 * info['price'] / 12
    elif info['price'] <= 625000 and percent < 0.05:
        PMI = 0.0085 * info['price'] / 12
    elif info['price'] >= 625000 and percent >= 0.05:
        PMI = 0.01 * info['price'] / 12
    elif info['price'] >= 625000 and percent < 0.05:
        PMI = 0.0105 * info['price'] / 12
    PITI = info['p_and_i'] - (info['p_and_i'] * (percent-0.035)) + (info['price'] / 1000) + PMI + 100 # Principal and Intrests from site, taxes (1% Rule in Seattle), PMI, homeowners insurance
    df.at[index, 'PITI'] = PITI
    HOA = info['hoa']
    if HOA == None:
        HOA = 0
    df.at[index, 'HOA'] = HOA
    df.at[index, 'total_monthly_payment'] = PITI + HOA
    
    # Gather rent info
    rent_list = list(info['rent_by_room'].values())
    min_room_rent = min(rent_list)
    projected_monthly_rental_income_live_out = sum(rent_list)
    df.at[index, 'projected_monthly_rental_income_live_out'] = projected_monthly_rental_income_live_out
    rent_list.remove(min_room_rent)
    projected_monthly_rental_income_live_in = sum(rent_list)
    df.at[index, 'projected_monthly_rental_income_live_in'] = projected_monthly_rental_income_live_in
    df.loc[index, 'reserves'] = reserves
    
    if info['type'] != 'Single-Family' or (info['type'] == 'Single-Family' and (option == 'conventional_min_assuming_boarder_income' or option == 'conventional_max_assuming_boarder_income')):
        # Calculate DTI if I live out of the home...
        post_purchase_DTI_live_out = (CURRENT_MONTHLY_DEBT + PITI + HOA) / (CURRENT_MONHTLY_GROSS_INCOME + (projected_monthly_rental_income_live_out * 0.75))
        # Calculate DTI if I live in the home...
        post_purchase_DTI_live_in = (CURRENT_MONTHLY_DEBT + PITI + HOA) / (CURRENT_MONHTLY_GROSS_INCOME + (projected_monthly_rental_income_live_in * 0.75))
    else: # Otherwise, live-in/live-out makes no dif here because is boarder income... (unless a special lender is found...)
        post_purchase_DTI_live_out = (CURRENT_MONTHLY_DEBT + PITI + HOA) / CURRENT_MONHTLY_GROSS_INCOME
        post_purchase_DTI_live_in = (CURRENT_MONTHLY_DEBT + PITI + HOA) / CURRENT_MONHTLY_GROSS_INCOME
    
    df.at[index, 'post_purchase_DTI_live_out'] = post_purchase_DTI_live_out
    df.at[index, 'post_purchase_DTI_live_in'] = post_purchase_DTI_live_in
    
    index += 1
    
is_valid_list = []
for i in range(len(df)):
    if df.loc[i, 'financing_option'] == 'fha':
        if df.loc[i, 'post_purchase_DTI_live_out'] <= 0.43 or df.loc[i, 'post_purchase_DTI_live_in'] <= 0.43:
            is_valid_list.append(True)
            continue
    elif df.loc[i, 'financing_option'] == '203k':
        if df.loc[i, 'post_purchase_DTI_live_out'] <= 0.43 or df.loc[i, 'post_purchase_DTI_live_in'] <= 0.43:
            is_valid_list.append(True)
            continue
    elif df.loc[i, 'financing_option'] == 'conventional_min' or df.loc[i, 'financing_option'] == 'conventional_max' or df.loc[i, 'financing_option'] == 'conventional_min_assuming_boarder_income' or df.loc[i, 'financing_option'] == 'conventional_max_assuming_boarder_income':
        if (df.loc[i, 'post_purchase_DTI_live_out'] <= 0.5 or df.loc[i, 'post_purchase_DTI_live_in'] <= 0.5) and info['type'] == 'Single-Family':
            is_valid_list.append(True)
            continue
    is_valid_list.append(False)

df.insert(0, 'is_valid', is_valid_list)

for i in range(len(df)):
    loan_amount = info['price'] - df.at[i, 'down_payment'] + (info['price'] * 0.05) # Includes closing costs HERE
    df.at[i, 'NWROI_live_in'] = str((((12 * df.at[i, 'projected_monthly_rental_income_live_in']) + (loan_amount / 30) - (12 * df.at[i, 'total_monthly_payment'])) / (df.loc[i, 'down_payment'] + info['upfront_renovation_costs'])) * 100) + '%'
    df.at[i, 'NWROI_live_out'] = str((((12 * df.at[i, 'projected_monthly_rental_income_live_out']) + (loan_amount / 30) - (12 * df.at[i, 'total_monthly_payment'])) / (df.loc[i, 'down_payment'] + info['upfront_renovation_costs'])) * 100) + '%'
    df.at[i, 'monthly_cash_flow_live_in'] = df.at[i, 'projected_monthly_rental_income_live_in'] - df.at[i, 'total_monthly_payment']
    df.at[i, 'monthly_cash_flow_live_out'] = df.at[i, 'projected_monthly_rental_income_live_out'] - df.at[i, 'total_monthly_payment']   
    df.at[i, 'monthly_cash_flow_after_reserves_live_in'] = df.at[i, 'projected_monthly_rental_income_live_in'] - df.at[i, 'total_monthly_payment'] - df.at[i, 'reserves']
    df.at[i, 'monthly_cash_flow_after_reserves_live_out'] = df.at[i, 'projected_monthly_rental_income_live_out'] - df.at[i, 'total_monthly_payment'] - df.at[i, 'reserves']
    df.at[i, 'yearly_cash_flow_live_in'] = df.at[i, 'monthly_cash_flow_after_reserves_live_in'] * 12
    df.at[i, 'yearly_cash_flow_live_out'] = df.at[i, 'monthly_cash_flow_after_reserves_live_out'] * 12
    df.at[i, 'monthly_cash_flow_plus_rent_savings_live_in'] = df.at[i, 'monthly_cash_flow_after_reserves_live_in'] + CURRENT_MONTHLY_RENT
    df.at[i, 'monthly_cash_flow_plus_rent_savings_live_out'] = df.at[i, 'monthly_cash_flow_after_reserves_live_out'] + CURRENT_MONTHLY_RENT
    df.at[i, 'total_monthly_net_income_after_mortgage_live_in'] = df.at[i, 'monthly_cash_flow_live_in'] + CURRENT_MONHTLY_NET_INCOME
    df.at[i, 'total_monthly_net_income_after_mortgage_live_out'] = df.at[i, 'monthly_cash_flow_live_out'] + CURRENT_MONHTLY_NET_INCOME
    
df
        

### TO DO ###
# Account for renovations in 203K
# Go back to 'info' and make sure all data is accounted for somewhere



,is_valid,financing_option,down_payment_percentage,down_payment,PITI,HOA,total_monthly_payment,projected_monthly_rental_income_live_out,projected_monthly_rental_income_live_in,reserves,...,monthly_cash_flow_live_in,monthly_cash_flow_live_out,monthly_cash_flow_after_reserves_live_in,monthly_cash_flow_after_reserves_live_out,yearly_cash_flow_live_in,yearly_cash_flow_live_out,monthly_cash_flow_plus_rent_savings_live_in,monthly_cash_flow_plus_rent_savings_live_out,total_monthly_net_income_after_mortgage_live_in,total_monthly_net_income_after_mortgage_live_out
0,False,fha,0.03500,14000.0,3783.333333,0.0,3783.333333,3540.0,2870.0,230.0,...,-913.333333,-243.333333,-1143.333333,-473.333333,-13720.0,-5680.0,-143.333333,526.666667,4086.666667,4756.666667
1,False,203k,0.03500,14000.0,3783.333333,0.0,3783.333333,3540.0,2870.0,230.0,...,-913.333333,-243.333333,-1143.333333,-473.333333,-13720.0,-5680.0,-143.333333,526.666667,4086.666667,4756.666667
2,False,conventional_min,0.05000,20000.0,3721.666667,0.0,3721.666667,3540.0,2870.0,230.0,...,-851.666667,-181.666667,-1081.666667,-411.666667,-12980.0,-4940.0,-81.666667,588.333333,4148.333333,4818.333333
3,False,conventional_min_assuming_boarder_income,0.05000,20000.0,3721.666667,0.0,3721.666667,3540.0,2870.0,230.0,...,-851.666667,-181.666667,-1081.666667,-411.666667,-12980.0,-4940.0,-81.666667,588.333333,4148.333333,4818.333333
4,False,conventional_max,0.07375,29500.0,3650.416667,0.0,3650.416667,3540.0,2870.0,230.0,...,-780.416667,-110.416667,-1010.416667,-340.416667,-12125.0,-4085.0,-10.416667,659.583333,4219.583333,4889.583333
5,False,conventional_max_assuming_boarder_income,0.07375,29500.0,3650.416667,0.0,3650.416667,3540.0,2870.0,230.0,...,-780.416667,-110.416667,-1010.416667,-340.416667,-12125.0,-4085.0,-10.416667,659.583333,4219.583333,4889.583333


In [6]:
print(df.loc[0])

is_valid                                                         False
financing_option                                                   fha
down_payment_percentage                                          0.035
down_payment                                                   14000.0
PITI                                                       3783.333333
HOA                                                                0.0
total_monthly_payment                                      3783.333333
projected_monthly_rental_income_live_out                        3540.0
projected_monthly_rental_income_live_in                         2870.0
reserves                                                         230.0
post_purchase_DTI_live_out                                    0.473175
post_purchase_DTI_live_in                                     0.502341
NWROI_live_in                                       17.15555555555557%
NWROI_live_out                                      70.75555555555556%
monthl